# Using stored parameters in DMF to simulate Flash with NRTL



### Note
>  This notebook requires the estimated NRTL parameters from the
>  [Parameter Estimation with DMF](../ParamEst/parameter_estimation_NRTL_using_state_block_solution.ipynb)
>  Jupyter notebook. If you have not already, please run that notebook first.

In this notebook, we will revisit the [Flash Unit Model](../../Basics/flash_unit_solution.ipynb) example but instead of using the ideal property package for the benzene-toluene system, we will be using the NRTL property package and we will also use the `binary interaction parameters` that were estimated in [Parameter Estimation with DMF](DMF_1_for_parameter_estimation_NRTL_using_unit_model_solution.ipynb). For the sake of brevity, this notebook will not demonstrate setting up the Flash Unit Model step by step. It is encouraged to first complete the stand alone Flash Unit Model notebook. 

## Key links to documentation
* Main IDAES online documentation page: https://idaes-pse.readthedocs.io/en/stable/index.html



In [ ]:
from pyomo.environ import ConcreteModel, SolverFactory, Constraint, value
from idaes.core import FlowsheetBlock

# Import idaes logger to set output levels
import idaes.logger as idaeslog

## Setup Data Management Framework (DMF)
We next connect to the DMF "workspace" where we stored the estimated parameters when we ran the
[Parameter Estimation with DMF](DMF_1_for_parameter_estimation_NRTL_using_unit_model_solution.ipynb)
Jupyter notebook.

In [ ]:
from idaes.dmf import DMF, magics
from pathlib import Path
idaes_home = Path("~/.idaes").expanduser()
if not idaes_home.exists():
    idaes_home.mkdir()
wspath = idaes_home / "workshop_workspace"
_dmf = DMF(wspath, create=True)

### Setup DMF "magics"
Initialize the Jupyter Notebook "magic" commands to enable things like `%dmf list`.
These commands all start with `%dmf`

In [ ]:
%dmf init ~/.idaes/workshop_workspace
%dmf list

In [ ]:
m = ConcreteModel()
m.fs = FlowsheetBlock(default={"dynamic": False})

## Define Properties

We need to define the property package for our flowsheet. In this example, we will be using the NRTL property package that is available as part of the IDAES framework. This property package supports ideal gas - ideal liquid, ideal gas - NRTL, and ideal gas - Wilson models for VLE. More details on this property package can be found at: https://idaes-pse.readthedocs.io/en/latest/model_libraries/core_library/property_models/activity_coefficient.html



In [ ]:
from idaes.generic_models.properties.activity_coeff_models.BTX_activity_coeff_VLE \
    import BTXParameterBlock

m.fs.properties = BTXParameterBlock(
    default={
        "valid_phase":('Liq', 'Vap'),
        "activity_coeff_model": 'NRTL'
    }
)

## Select estimated NRTL parameter split to use

In [ ]:
# Select estimated NRTL parameter split to use
from ipywidgets import widgets
w = widgets.Dropdown(options=["1", "2"], value="1", description="Choose estimated parameter split",
                    style={"description_width": "initial"})
display(w)
use_split = w.value

In the following cell, we create a method to fix the binary interaction parameters from the DMF. 

In [ ]:
def NRTL_model(model, data):
    props = model.fs.properties
    # Fix NRTL specific variables
    # alpha values (set at 0.3)
    props.alpha["benzene", "benzene"].fix(0)
    props.alpha["benzene", "toluene"].fix(0.3)
    props.alpha["toluene", "toluene"].fix(0)
    props.alpha["toluene", "benzene"].fix(0.3)

    # initial tau values
    tau = data["parameters"]["tau"]
    props.tau["benzene", "benzene"].fix(0)
    props.tau["benzene", "toluene"].fix(tau["benzene,toluene"])
    props.tau["toluene", "toluene"].fix(0)
    props.tau["toluene", "benzene"].fix(tau["toluene,benzene"])

    # Set bounds on variables to be estimated
    props.tau["benzene", "toluene"].setlb(-5)
    props.tau["benzene", "toluene"].setub(5)

    props.tau["toluene", "benzene"].setlb(-5)
    props.tau["toluene", "benzene"].setub(5)

# Find & load NRTL parameters for the chosen split in the DMF
records = _dmf.find_one(name=f"BT NRTL est param{use_split}")
if records:
    # Create NRTL model with parameters
    NRTL_model(m, records.data)
else:
    print("No data found in DMF: Abort")
    exit(0)

## Adding Flash Unit

Now that we have the flowsheet and the properties defined, we can create the flash unit and add it to the flowsheet. 


In [ ]:
from idaes.generic_models.unit_models import Flash
m.fs.flash = Flash(default={"property_package": m.fs.properties})

In [ ]:
# Display the values fixed for the binary interaction parameters from the DMF
m.fs.properties.tau.display()

## Set Operating Conditions


In [ ]:

from idaes.core.util.model_statistics import degrees_of_freedom
print("Degrees of Freedom =", degrees_of_freedom(m))

In [ ]:
# Inlet specifications given above
m.fs.flash.inlet.flow_mol.fix(1)
m.fs.flash.inlet.temperature.fix(368)
m.fs.flash.inlet.pressure.fix(101325)
m.fs.flash.inlet.mole_frac_comp[0, "benzene"].fix()
m.fs.flash.inlet.mole_frac_comp[0, "toluene"].fix(0.5)


m.fs.flash.heat_duty.fix(0)
m.fs.flash.deltaP.fix(0)

In [ ]:

print("Degrees of Freedom =", degrees_of_freedom(m))

## Initializing the Model


In [ ]:
m.fs.flash.initialize(outlvl=idaeslog.INFO)

In [ ]:
solver = SolverFactory('ipopt')
status = solver.solve(m, tee=True)

## Viewing the Results


In [ ]:
# Print the pressure of the flash vapor outlet
print('Pressure =', value(m.fs.flash.vap_outlet.pressure[0]))

print()
print('Output from display:')
# Call display on vap_outlet and liq_outlet of the flash
m.fs.flash.vap_outlet.display()
m.fs.flash.liq_outlet.display()

In [ ]:
m.fs.flash.report()